In [11]:
import dotenv
import os
from langchain_community.chat_models.openai import ChatOpenAI
from discussion_agents.cog.functional.critic import _prompt_agent, _prompt_critique
from discussion_agents.cog.prompts.critic import HOTPOTQA_FEWSHOT_EXAMPLES_COT, HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC

dotenv.load_dotenv("../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(openai_api_key=openai_api_key)

In [12]:
question = 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring'

answer = _prompt_agent(
    llm=llm,
    question=question,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_COT
)

In [13]:
answer

"Let's break it down step by step. The individual in question is Badr Hari. Badr Hari was once considered the best kickboxer in the world. However, he has been involved in controversies related to his unsportsmanlike conduct in the sport and crimes of violence outside of the ring. So the answer is: Badr Hari."

In [14]:
critique = _prompt_critique(
    llm=llm,
    question=question,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC,
    answer=answer
)

In [16]:
print(critique)

The question asks for the name of an individual, and the answer "Badr Hari" is a name. So it seems plausible. 

2. Truthfulness:

Let's search the question in Google:

> Search Query: Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring
> Evidence: [Badr Hari - Wikipedia] Badr Hari is a Moroccan-Dutch super heavyweight kickboxer from the Netherlands.

The evidence supports that Badr Hari fits the description provided in the question.

Therefore, the proposed answer is both plausible and truthful.

Question: Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring
Here's the most possible answer: Badr Hari.


In [23]:
from discussion_agents.cog.prompts.critic import CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA

In [24]:
a = CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA.format(examples=HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC, question=question, answer=answer)

In [30]:
print(critique)

The question asks for the name of an individual, and the answer "Badr Hari" is a name. So it seems plausible. 

2. Truthfulness:

Let's search the question in Google:

> Search Query: Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring
> Evidence: [Badr Hari - Wikipedia] Badr Hari is a Moroccan-Dutch super heavyweight kickboxer from the Netherlands.

The evidence supports that Badr Hari fits the description provided in the question.

Therefore, the proposed answer is both plausible and truthful.

Question: Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring
Here's the most possible answer: Badr Hari.


: 

In [27]:
print(a + critique)

Question: Serianna is a band of what genre that combines elements of heavy metal and hardcore punk?
Proposed Answer: Let's think step by step. Serianna is a band of metalcore genre. Metalcore is a subgenre of heavy metal and hardcore punk. So Serianna is a band of heavy metal and hardcore punk. So the answer is: heavy metal and hardcore punk.

What's the problem with the above answer?

1. Plausibility:

The question asks for the genre that combines elements of heavy metal and hardcore punk, and the answer is "heavy metal and hardcore punk", simply repeat the question. So it's not plausible.

2. Truthfulness:

Let's search the question in google:
> Search Query: Serianna is a band of what genre that combines elements of heavy metal and hardcore punk? site: wikipedia.org
> Evidence: [Metalcore - Wikipedia] Metalcore is a fusion music genre that combines elements of extreme metal and hardcore punk.

The evidence suggests that metalcore is a genre that combines elements of extreme metal an

In [22]:
_, search_query = critique.split("> Search Query:")[:2]
search_query = search_query.split("\n")[0].strip()

In [ ]:
if "> Search Query: " in critique:
    pass
elif "most possible answer: " in critique:
    pass
else:
    pass